# Topic Clustering with LDA and Gaussian LDA
*Zijian Chen zc2386*  
*Yiran Wang yw3201*  
*Columbia University*
## 1. Introduction
**GOAL**  
The goal in modeling a large corpus of discrete documents is to find the latent clustering pattern of those documents as we assume that each document has a probability distribution over certain number of topics, and the words in each documents are drawn from documents' topic distribution. It is hard because we are not sure about the word distribution whether they are discrete or continuous, nor do we have any information about the number of topics in many cases.  
**DATA**  
To solve topic clustering problem we access [NIPS](https://cs.nyu.edu/~roweis/data.html) corpus data.

In [1]:
import sys
import numpy as np
import tensorflow as tf
import edward as ed
import glob
import pickle
import time

if '../' not in sys.path:
    sys.path.append('../')

from models.lda import LDA
from models.lda import GaussianLDA


## 2. LDA Model 

### 2.1 Modeling [Blei et al. (2003)](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)
LDA assumes the words in every documents are exchangeable, and each of which is a multinomial distribution over vocabularies of size $V$. The likelihood of a corpus $\mathbf{D}$ contains $D$ documents is given as following:  
\begin{align}
	p(\mathbf{D}; \alpha, \beta)=p(\theta, \beta, z, w)&=\prod_{k=1}^{K} p(\beta_{k};\eta)\prod_{d=1}^{D} p(\theta_{d} ; \alpha)(\prod_{n=1}^{N}p(z_{dn} | \theta_{d})p(w_{dn} |z_{dn},\beta_{z_{dn}})\\
	&= \prod_{k=1}^{K} p(\beta_{k};\eta)\prod_{d=1}^{D}p(\theta_{d} ; \alpha)(\prod_{n=1}^{N}\theta_{z_{dn}}f(w_{dn};\beta_{z_{dn}}))
\end{align}
    
**Nodes/Parameter description:** 
* $\alpha, \eta$: dirichlet hyper parameter.  
* $\theta_{d}$: global latent variable, per-doc topic proportions; $\theta_{d}$ $\in$ $R^{k}$. $k$ dimensional Dir. random variable $\theta$ can take values in (k-1)-simplex with $p.d.f$ $p(\theta | \alpha)=\dfrac{\Gamma(\sum_{i=1}^{k}\alpha_{i})}{\prod_{i=1}^{k}\Gamma(\alpha_{i})}\theta_{1}^{\alpha_{1}-1} \cdots \theta_{k}^{\alpha_{k}-1} $.
* $Z_{dn}$: local latent variables, topic assignment drawn from distribution $\theta_{d}$, which takes values from {1, 2 .... k}
* $\beta_{z_{dn}}$: global latent variables, topic's distribution over words; per-corpus topic distribution.
* $W_{dn}$: observed variables, $(v-1)-$simplex, where $v$ is the vocabulary size. Observed $n^{th}$ word from $d^{th}$ document.

As shown in the graph below, the **LDA generative process** is as following:  
1. For each topic $k \in \{1, 2, ..., K\}$, draw $\beta_{k} \sim Dirichlet(\eta)$  
2. For each document $d \in \{1, 2, ...,D\}$:  
    a) Draw $\theta_{d} \sim Dirichlet (\alpha)$  
    b) For each word $w_{n}$, $n \in \{1, 2, ..., N_{d}\}$ in document $d$:  
    * Draw $z_{dn} \sim Categorical(\theta_{d})$.  
    * Generate word $w_{dn}\sim p(w_{dn} | z_{dn}, \beta_{z_{dn}})$, a categorical probability conditioned on the topic $z_{dn}$.  

<img src="LDABlei.jpg" width="600"/>

### 2.2 Collapsed Gibbs Sampling Inference for LDA (MCMC) 

We planned to implement the **Black Box variational inference** method (klqp method) in Edward at the very beginning, however if we include all parameters $\theta$, $z$, and $\beta$ for the posterior inference, the result cannot converge. Because klqp in Edward has poor performance at approximating the Dirichlet distribution, especially if we have full-rank covariances for the mixture components. This problem is partially due to the high variance in stochastic optimization if only implementing the vanilla version of BBVI.  
In addition, because of the slow convergence of **Gibbs Sampling** inference of LDA, we implemented the **Collapsed Gibbs Sampling** Inference instead [Griffiths et al. (2004).](http://www.pnas.org/content/pnas/101/suppl_1/5228.full.pdf). Since we are only interested in the posterior distribution over the assignment of words in $k$ topics respectively, $p(\mathbf{z}|\mathbf{w})$, we can have inexplicit expression of the posterior estimation of $\theta$ and $\beta$.   
In Monte Carlo Markov Chain, we construct a Markov chain on latent variables which converges to a stationary distribution. This would be used for the estimation of the posterior distribution. In order to draw sample at the time t, we need to construct the complete conditional distribution of latent variables, $p(z_{d,i}|\mathbf{z}_{d,-i}, \mathbf{w})$ in this algorithm. Taking the advantage of the conjugate conditional family, the probability distribution can be expressed as: 
\begin{align}
 	p(z_{d,i}=j|\mathbf{z}_{d,-i}, \mathbf{w}) & \propto \frac{n_{-i,j}^{(w_{d,i}) }+ \eta}{n_{-i,j}^{(.)} + V\eta} \frac{n_{-i,j}^{(d)}+\alpha}{n_{-i,.}^{(d)}+k\alpha}
\end{align}
The first fraction express the probability of $w_{d,i}$ under topic $j$, and the second term expresses the probability of topic $j$ document $d_{i}$. The notations are defined as follows:
* $n_{-i,j}^{(w_{d,i})}$ is the number of times this $i^{th}$ observed word in document $d$ is assigned to topic $j$ while not including the current assignment $z_{d,i}$.  
* $n_{-i,j}^{(.)}$ is the number of times any word is assigned to topic $j$ while not including the current assignment $z_{d,i}$.  
* $n_{-i,j}^{(d)}$ is the number of times a word from document $d$ is assigned to topic $j$ while not including the current assignment $z_{d, i}$.
* $n_{-i,.}^{(d)}$ is the number words in document $d$ while not including the current assignment $z_{d, i}$.
* $V$ is the vocabulary size, and $k$ is the number of topics defined at the very beginning.  

With the set of samples drawn after burn-in period, we can evaluate $\theta$ and $\beta$ by integrating across the full set of samples as follows:
 \begin{align}
 	\hat{\beta_{j}^{(w)}} &= \frac{n_{j}^{(w)} + \eta}{n_{j}^{(.)}+V\eta}\\
 	\hat{\theta_{j}^{(d)}} &= \frac{\theta_{j}^{(d)} + \alpha}{n_{.}^{(d)}+k\alpha}
 \end{align}

### 2.3 Criticism for LDA

**Experiment**  
Due to the duration of running time, we run our model of 2-year NIPS documents (year 2012 and year 2011) and for each document, we extracted the abstract part which summarizes the entire document. Furthermore, we did the NLP preprocessing of the input: removed punctuation and the stop words, changed words into lowercase, and stemmed the words with [ntlk](https://www.nltk.org/). We initialize the hyper-parameters $\alpha = 0.1$ and $\eta = 0.01$. There are nine major academic subject areas for NIPS: *Algorithm*, *Application*, *Data Competition, Implementation and Software*, *Deep Learning*, *Neuroscience and Cognitive science*, *Optimization*, *Probabilistic Methods*, *Reinforcement Learning & Planning*, and *Theory*, therefore we set $k=9$.

In [6]:
t1 = time.time()
datafile = "../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_*.txt"
txt_files = glob.glob(datafile)
D = len(txt_files)  # number of documents
print("number of documents, D: {}".format(D))

N = [0] * D  # words per doc
K = 9  # number of topics
T = 300 # samples drawn in the burnin period
S = 200 # samples collected for collapsed gibbs sampling

wordIds = [None] * D
count = 0  # count number of documents
for file in (txt_files):
    with open(file, 'rt', encoding="ISO-8859-1") as f:
        wordIds[count] = list(map(int, f.readline().split()))
        N[count] = len(wordIds[count])
        wordIds[count] = np.array(wordIds[count]).astype('int32')
    print("load" + file + "finished")
    count += 1
IdtoWord = {}
vocab = set()

with open("../DataPreprocess/wordToID_short_12_20.txt") as f:
    for line in f:
        line = line.split()
        IdtoWord[int(line[1])] = line[0]
        vocab.add(line[0])
V = len(vocab)  # vocabulary size21
tokens = [None] * V
for key in IdtoWord:
    tokens[key] = IdtoWord[key]
print("vocab size is {}".format(V))
print("load wordToID finished")

number of documents, D: 20
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0024.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0031.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0136.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0122.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0080.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0096.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0108.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0045.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0052.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0103.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0089.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0129.txtfinished
load../DataPreprocess/nipstxt/nipstoy20/doc_short_wID_12_0115.txtfinished
load../Data

**PMI Evaluation for Top Words**  [Bouma (2009)](https://svn.spraakdata.gu.se/repos/gerlof/pub/www/Docs/npmi-pfd.pdf)  
Point-wise mutual information (PMI) is a measure of how much the actual probability of a particular co-occurrence of events $p(x, y)$ differs from what we would expect it to be on the basis of the probabilities of the individual events $p(x)$ and $p(y)$. Also, the calculation based on the independent assumption of $x$ and $y$ is given as:
\begin{align}
 	pmi (x, y) = \ln \frac{p(x,y)}{p(x)p(y)}
\end{align} 
The joint probability is estimated by the time we see both words in a 20-length sliding window over the entire corpus. The results are shown here,

In [11]:
model = LDA(K, V, D, N)
print("model constructed")
model.collapsed(wordIds, S, T, tokens)
print("inference finished")

model constructed
burnin: 0
burnin: 1
burnin: 2
burnin: 3
burnin: 4
burnin: 5
burnin: 6
burnin: 7
burnin: 8
burnin: 9
burnin: 10
burnin: 11
burnin: 12
burnin: 13
burnin: 14
burnin: 15
burnin: 16
burnin: 17
burnin: 18
burnin: 19
burnin: 20
burnin: 21
burnin: 22
burnin: 23
burnin: 24
burnin: 25
burnin: 26
burnin: 27
burnin: 28
burnin: 29
burnin: 30
burnin: 31
burnin: 32
burnin: 33
burnin: 34
burnin: 35
burnin: 36
burnin: 37
burnin: 38
burnin: 39
burnin: 40
burnin: 41
burnin: 42
burnin: 43
burnin: 44
burnin: 45
burnin: 46
burnin: 47
burnin: 48
burnin: 49
burnin: 50
burnin: 51
burnin: 52
burnin: 53
burnin: 54
burnin: 55
burnin: 56
burnin: 57
burnin: 58
burnin: 59
burnin: 60
burnin: 61
burnin: 62
burnin: 63
burnin: 64
burnin: 65
burnin: 66
burnin: 67
burnin: 68
burnin: 69
burnin: 70
burnin: 71
burnin: 72
burnin: 73
burnin: 74
burnin: 75
burnin: 76
burnin: 77
burnin: 78
burnin: 79
burnin: 80
burnin: 81
burnin: 82
burnin: 83
burnin: 84
burnin: 85
burnin: 86
burnin: 87
burnin: 88
burnin: 89
bu

In [15]:
print(time.time() - t1)
model.getTopWords(tokens)
comatrix = pickle.load(open("../20abstract1yearAll.pickle", "rb"))
comatrix += 1
model.getPMI(comatrix)

707.7561919689178
topic 0
network 0.12638986
model 0.0884915
attractor 0.05059373
select 0.03164481
output 0.02532846
associ 0.02532846
signal 0.02532846
experi 0.02532846
law 0.01901216
discuss 0.01901216
topic 1
gener 0.05228643
process 0.04648335
find 0.034876928
recognit 0.029073752
complex 0.029073752
experiment 0.029073752
base 0.023270622
observ 0.023270622
imag 0.023270622
suggest 0.023270622
topic 2
languag 0.094329216
acquisit 0.047203988
task 0.047203988
boundari 0.031495456
form 0.031495456
recurr 0.031495456
classif 0.02364116
coeffici 0.02364116
object 0.02364116
exampl 0.02364116
topic 3
effect 0.058758754
respons 0.058758754
also 0.044087414
stimulu 0.03675175
channel 0.029416082
differenti 0.029416082
given 0.022080416
compar 0.022080416
formul 0.022080416
ground 0.022080416
topic 4
show 0.094742015
result 0.072895296
figur 0.072895296
ep 0.04376643
propos 0.029201943
stop 0.029201943
simul 0.021919563
higher 0.021919563
recent 0.021919563
evolv 0.021919563
topic 5
neu

#### Our results: ###
<img src="LDA2y_results.jpg" width="950"/>

## 3. Gaussian LDA Model 

### 3.1 Modeling  [Das et al. (2015)](http://www.aclweb.org/anthology/P15-1077) 
Instead of generating words from categorical distribution, Gaussian LDA integrates the word embedding that aims at capturing more semantic regularities in language and generates words from multivariate Gaussian distributions. The observed words input are pre-trained word2vec embedding vectors with fixed dimensions [gensim](https://radimrehurek.com/gensim/models/word2vec.html). Our model used 25 dimensional word embeddings instead of the pre-defined 100 dimension due to the feasibility and running time while implementing on Edward.  
The likelihood of a corpus $\mathbf{D}$ contains $D$ documents is given as following:
\begin{align}
p(\mathbf{D}; \alpha, \beta)=\prod_{k=1}^{K} p(\Sigma_{k}|\Psi, v)p(\mu_{k}|\mu_{0}, \Sigma_{k})\prod_{d=1}^{D} p(\theta_{d} | \alpha)(\prod_{n=1}^{N}p(z_{dn} | \theta_{d})p(w_{dn} |z_{dn},\Sigma_{z_{dn}},\mu_{z_{dn}}))
\end{align}

**Nodes Decription**  
* $ \psi, v$: Inverse Wishart distribution hyper parameter	
* $\mu_{0}$: Gaussian distribution hyper parameter	
* $\Sigma_{z_{dn}}$: global latent variable, per-topic covariance matrix for Multivariate Gaussian distribution of words.	
* $\mu_{z_{dn}}$: global latent variable, per-topic mean vector for Multivariate Gaussian distribution of words.  

**Gaussian-LDA generative process:** 
1. For each topic $k \in \{1, 2, ..., K\}$:  
    a) Draw topic covariance matrix $\Sigma_{k} \sim W(\Psi, v)$  
    b) Draw topic mean $\mu_{k} \sim N(\mu_{0}, I_{k})$  
2. For each document $d \in \{1, 2, ...,D\}$ in corpus:  
    a) Draw topic distribution $\theta_{d} \sim Dirichlet (\alpha)$  
    b) For each word $w_{n}$, $n \in \{1, 2, ..., N_{d}\}$ in document $d$:    
    * Draw $z_{dn} \sim Categorical(\theta_{d})$.  
    * Draw word $w_{dn}\sim N(\mu_{z_{dn}}, \Sigma_{z_{dn}})$. 

<img src="GaussianLDA.png" width="400"/>

### 3.2 Black Box Variational Inference for Gaussian LDA  
Paper [Das et al. (2015)](http://www.aclweb.org/anthology/P15-1077) provides the Collapsed Gibbs Sampling posterior inference method for Gaussian LDA, however Collapsed Gibbs Sampling does not work with mixture random variables in Edward. With the blessing of the conjugate priors for topic proportions, we can integrate them out during the inference as we got rid of the Dirichlet distribution in klqp implementation in Edward. The variational distribution for $\mu$ and $\Sigma$ are that same distribution as the prior while taking the independence assumption.

In [16]:
t2 = time.time()
datafile = "../DataPreprocess/nips12we25/short_wordembed_*.txt"
txt_files = glob.glob(datafile)
D = len(txt_files)  # number of documents
print("number of documents, D: {}".format(D))
N = [0] * D  # words per doc
K = 9  # number of topics
T = 2500 # number of iteration
S = 5 # # of sample 
wordIds = [None] * D
count = 0  # count number of documents
for file in (txt_files):
    with open(file, 'rt', encoding="ISO-8859-1") as f:
        vec = []
        for line in f:
            vec.append(list(map(float, line.split())))
        N[count] = len(vec)
        wordIds[count] = vec
    print("load" + file + "finished")
    count += 1
nu = len(wordIds[0][0])
print("dimension:", nu)

wordToId = dict()
tokens = []
# with open("DataPreprocess/wordToID_short_12.txt") as f:
with open("../DataPreprocess/wordToID_short_12_20.txt") as f:
    for line in f:
        line = line.split()
        wordToId[line[0]] = len(tokens)
        tokens.append(line[0])
V = len(tokens)  # vocabulary size21
print("vocab size is {}".format(V))
print("load wordId finished")

wordVec = [None] * V
# with open("DataPreprocess/word_vectors_25.txt") as f:
with open("../DataPreprocess/we_12_20.txt") as f:
    for line in f:
        line = line.split()
        if line[0] in wordToId:
            wordVec[wordToId[line[0]]] = list(map(float, line[1:]))
print("load word embeddings finished")
D = 20

model = GaussianLDA(K, D, N, nu)
print("model constructed")
model.klqp(wordIds, S, T, wordVec)
print("inference finished")

number of documents, D: 150
load../DataPreprocess/nips12we25/short_wordembed_0526.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0724.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0687.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0136.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0122.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0491.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0321.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0335.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0484.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0645.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0862.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0731.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0080.txtfinished
load../DataPreprocess/nips12we25/short_wordembed_0533.txtfinished
load../DataPreprocess/nips12we25/short_wordembed

model constructed
2500/2500 [100%] ██████████████████████████████ Elapsed: 848s | Loss: 41379.480
inference finished


### 3.3 Criticism for Gaussian LDA 
The top words for each topic and corresponding pmi is shown in the following chart. We can see improvement in pmi score as the average pmi changes from 0.203 to 0.600 with more topic interpretability.

In [24]:
print(time.time() - t2)
model.getTopWords(wordVec, tokens)
print("get top words finished")
print(time.time() - t2)
comatrix = pickle.load(open("../20abstract1yearAll.pickle", "rb"))
# comatrix = pickle.load(open("DataPreprocess/comatrix1y.pickle", "rb"))
comatrix += 1 # normalize
model.getPMI(comatrix)
print(time.time() - t2)

1267.902379989624
topic 0
agreement -40.500427
fundament -40.536804
diffus -40.77791
sequenti -40.988068
green -41.114006
manner -41.33602
event -42.39844
perceptu -42.421795
carri -42.865368
excit -42.986515
usual -43.21483
neither -43.373848
psychophys -43.553295
come -43.810966
establish -43.97314
topic 1
recent -32.83008
first -32.936684
demonstr -32.970604
procedur -33.01648
simpl -33.033684
develop -33.06268
framework -33.092064
simul -33.116978
experiment -33.139137
context -33.273876
provid -33.274765
includ -33.295822
similar -33.317406
theori -33.326797
well -33.344704
topic 2
encod -46.534046
global -46.546097
locat -46.570942
axi -46.5823
certain -46.58876
strength -46.596672
overlap -46.605545
top -46.610847
sensor -46.64315
phase -46.643295
except -46.64789
region -46.6502
synchroni -46.66201
boundari -46.67524
synapt -46.677597
topic 3
primari -9.362706
distinct -9.379214
inter -9.392682
intermedi -9.393285
agre -9.398555
subpopul -9.401191
inconsist -9.415439
bias -9.45

#### Our results: ####
<img src="GLDA50v3.jpg" width="950"/>

### *References*
* [Blei, D. M., Ng, A. Y., and Jordan, M. I. (2003).](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) Latent dirichlet allocation. Journal of machine Learning research, 3(Jan):993–1022. 
* [Bouma, G. (2009).](https://svn.spraakdata.gu.se/repos/gerlof/pub/www/Docs/npmi-pfd.pdf) Normalized (pointwise) mutual information in collocation extraction. Proceedings of GSCL, 31-40.
* [Das, R., Zaheer, M., and Dyer, C. (2015).](http://www.aclweb.org/anthology/P15-1077) Gaussian lda for topic models with word embeddings. In Proceedings of the 53rd Annual Meeting of the Association for Computational Linguistics and the 7th International Joint Conference on Natural Language Processing (Volume 1: Long Papers), volume 1, pages 795–804.
* [Griffiths, T. L., & Steyvers, M. (2004).](http://www.pnas.org/content/pnas/101/suppl_1/5228.full.pdf) Finding scientific topics. Proceedings of the National academy of Sciences, 101(suppl 1), 5228-5235.
